<a href="https://colab.research.google.com/github/vijayk2000/capstone_project/blob/main/Pinacle_capstone_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain_chroma
%pip install langchain_openai
%pip install langchain_google_genai
%pip install langchain_huggingface
%pip install langchain_community
%pip install FlagEmbedding
!pip install python-dotenv
%pip install pypdf

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.docstore.document import Document

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the files and index them in a vector database


In [4]:
docs= []
for doc in PyPDFDirectoryLoader("/content/data").load():


    metadata ={
        "source": doc.metadata["source"],
        "page": doc.metadata["page"],

    }
    doc_text = ' '.join(doc.page_content.split())
    # Remove newlines and extra spaces
    docs.append(Document(page_content=doc_text,
                                   metadata=metadata))

# PyPDFDirectoryLoader loads all PDFs in a directory

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
from langchain_chroma import Chroma
from FlagEmbedding import BGEM3FlagModel
# The splitting and chunking strategy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv(".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACE_TOKEN')

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [11]:
# Download the embeddings
open_ai_large = OpenAIEmbeddings(model="text-embedding-3-large")
open_ai_small = OpenAIEmbeddings(model="text-embedding-3-small")


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
def create_chroma_db(docs, embedding_list, embedding_names, similarity_type="cosine"):
  splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  chunked_docs = splitter.split_documents(docs)

  for embedding, name in zip(embedding_list, embedding_names):
    for similarity in similarity_type:
      persist_dir = f"./capstone_db_{name}_{similarity}"
      chromba_db = Chroma.from_documents(documents=chunked_docs,
                                        collection_name=f'capstone_db_{name}_{similarity}',
                                        embedding=embedding,
                                        collection_metadata={"hnsw:space": similarity},
                                         persist_directory=persist_dir)

      print(f"Chroma DB created for embedding: {name} with similarity: {similarity} in directory: {persist_dir}")

In [13]:
docs = docs
embedding_list = [open_ai_large, open_ai_small, hf]
embedding_names = ["open_ai_large", "open_ai_small","mp_allnet"]
similarity_type = ["cosine", "l2"]

In [14]:
create_chroma_db(docs=docs, embedding_list=embedding_list, embedding_names=embedding_names, similarity_type=similarity_type)

Chroma DB created for embedding: open_ai_large with similarity: cosine in directory: ./capstone_db_open_ai_large_cosine
Chroma DB created for embedding: open_ai_large with similarity: l2 in directory: ./capstone_db_open_ai_large_l2
Chroma DB created for embedding: open_ai_small with similarity: cosine in directory: ./capstone_db_open_ai_small_cosine
Chroma DB created for embedding: open_ai_small with similarity: l2 in directory: ./capstone_db_open_ai_small_l2
Chroma DB created for embedding: mp_allnet with similarity: cosine in directory: ./capstone_db_mp_allnet_cosine
Chroma DB created for embedding: mp_allnet with similarity: l2 in directory: ./capstone_db_mp_allnet_l2


In [15]:
# embedding_name = []
# similarity_types = []
# retriever_details = []

In [16]:
# for embedding, name in zip(embedding_list, embedding_names):

#   for similarity  in similarity_type:

#     chroma_db = Chroma(persist_directory=f"./capstone_db_{name}_{similarity}",
#                         collection_name=f'capstone_db_{name}_{similarity}',
#                          embedding_function=embedding)

#     similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
#                                                 search_kwargs={"k": 5, "score_threshold": 0.0})
#     embed_search = similarity_retriever.invoke("what is attention layer", k=5)
#     embedding_name.append(name)

#     retriever_details.append(embed_search)
#     similarity_type.append(similarity)


# print((embedding_name, similarity_type))

In [17]:
embedding_list = [open_ai_large, open_ai_small, hf]
embedding_names = ["open_ai_large", "open_ai_small","mp_allnet"]
similarity_type = ["cosine", "l2"]

In [18]:
# embedding_names =[]
# retriever_details = []
# similarity_types = []
combined_list_results = []

def find_best_embedding(embedding_list, embedding_names, similarity_type, question):
  # load from disk
  results = []

  for embedding, name in zip(embedding_list, embedding_names):
    for similarity  in similarity_type:
      print(embedding)
      print(name)
      print(similarity)

      chroma_db = Chroma(persist_directory=f"./capstone_db_{name}_{similarity}",
                        collection_name=f'capstone_db_{name}_{similarity}',
                         embedding_function=embedding)

      similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                search_kwargs={"k": 5, "score_threshold": 0.0})
      embed_search = similarity_retriever.invoke(question, k=5)

    combined_list_results.append({
        "embedding_name": name,
        "similarity_type": similarity,
        "retriever_details": embed_search
    })

  return combined_list_results

## Analyze Cosine  Similarity

In [19]:
ques="what is attention layer"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names,
                              similarity_type = similarity_type, question = ques)
print(output)

client=<openai.resources.embeddings.Embeddings object at 0x7e5d6a07d310> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e5d69929690> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
open_ai_large
cosine
client=<openai.resources.embeddings.Embeddings object at 0x7e5d6a07d310> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e5d69929690> model='text-embedding-3-large' dimensions=

## Analyzing Cosine Similarity results

In [20]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "cosine"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: cosine
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all

In [21]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "l2"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: l2
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all of 

## Cosine similarity is better than Euclidean similarity.

In [22]:
ques="what is gemini"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names,
                              similarity_type = similarity_type, question = ques)
#print(output)

client=<openai.resources.embeddings.Embeddings object at 0x7e5d6a07d310> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e5d69929690> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
open_ai_large
cosine
client=<openai.resources.embeddings.Embeddings object at 0x7e5d6a07d310> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e5d69929690> model='text-embedding-3-large' dimensions=

In [23]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "cosine"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: cosine
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all

In [24]:
ques="what are the conclusions of llms"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names,
                              similarity_type = similarity_type, question = ques)
#print(output)

client=<openai.resources.embeddings.Embeddings object at 0x7e5d6a07d310> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e5d69929690> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
open_ai_large
cosine
client=<openai.resources.embeddings.Embeddings object at 0x7e5d6a07d310> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e5d69929690> model='text-embedding-3-large' dimensions=

In [25]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "cosine"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: cosine
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all

## Creating a RAG based model

In [98]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.docstore.document import Document
from langchain import hub
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage

In [29]:
open_ai_large = OpenAIEmbeddings(model="text-embedding-3-large")

In [33]:
docs= []
for doc in PyPDFDirectoryLoader("/content/data").load():


    metadata ={
        "source": doc.metadata["source"],
        "page": doc.metadata["page"],

    }
    doc_text = ' '.join(doc.page_content.split())
    # Remove newlines and extra spaces
    docs.append(Document(page_content=doc_text,
                                   metadata=metadata))

# PyPDFDirectoryLoader loads all PDFs in a directory

In [31]:
# The vectorstore we'll be using
from langchain_chroma import Chroma

# The embedding engine that will convert our text to vectors
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [39]:
splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)

Create Chroma DB and embedding

In [61]:
chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name='capstone_db_rag',
                                  collection_metadata={"hnsw:space":"cosine"},
                                  persist_directory="./capstone_db_rag",
                                  embedding=open_ai_large)

In [71]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                             search_kwargs={"k": 5, "score_threshold": 0.2})

In [73]:
# similarity_retriever.invoke("what is gemini", k=5)

In [74]:
prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point. also show the top 3 context documents of the answer.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [27]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [75]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
       |
    prompt_template
      |
    chatgpt
)

In [85]:
from IPython.display import display, Markdown
query = "what is a LLM"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))


/usr/local/lib/python3.11/dist-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='ae04628c-934a-48c5-8f6b-3526c1c158f2', metadata={'page': 3, 'source': '/content/data/mistral_paper.pdf'}, page_content='Figure 4: Performance of Mistral 7B and different Llama models on a wide range of benchmarks. All models were re-evaluated on all metrics with our evaluation pipeline for accurate comparison. Mistral 7B significantly outperforms Llama 2 7B and Llama 2 13B on all benchmarks. It is also vastly superior to Llama 1 34B in mathematics, code generation, and reasoning benchmarks. Model Modality MMLU HellaSwag WinoG PIQA Arc-e Arc-c NQ TriviaQA HumanEval MBPP MATH GSM8K LLaMA 2 7B Pretrained 44.4% 77.1% 69.5% 77.9% 68.7% 43.2% 24.7% 63.8% 11.6% 26.1% 3.9% 16.0% LLaMA 2 13B Pretrained 55.6% 80.7% 72.9% 80.8% 75.2% 48.8% 29.0% 69.6% 18.9% 35.4% 6.0% 34.3% Code-Llama 7B Finetuned 36.9% 62.9% 62.3% 72.8% 59.4% 34.5% 11.0% 34.9

A LLM, or Large Language Model, is a type of artificial intelligence model designed to understand and generate human language by processing vast amounts of text data.

Top 3 context documents:
1. Document discussing the definition and capabilities of LLMs.
2. Document explaining the architecture and training methods of LLMs.
3. Document outlining applications and use cases of LLMs in various fields.

In [91]:
rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is."""

rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)


In [95]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks.
                      Use the following pieces of retrieved context to answer the question.
                      If you don't know the answer, just say that you don't know.
                      Keep the answer upto 5 lines unless the user asks for more information

                      Context:
                      {context}
                  """

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(chatgpt, qa_prompt)

qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [102]:
chat_history = []

question = "what is mistral 7b?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])

Mistral 7B is a 7-billion-parameter language model designed for high performance and efficiency in natural language processing tasks. It outperforms other models, including the Llama 2 13B, across various benchmarks. Mistral 7B utilizes advanced attention mechanisms like grouped-query attention and sliding window attention to enhance inference speed and handle longer sequences effectively. It is released under the Apache 2.0 license and is suitable for a wide range of applications.


In [105]:
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response['answer'])])

chat_history

[HumanMessage(content='what is mistral 7b?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Mistral 7B is a 7-billion-parameter language model designed for high performance and efficiency in natural language processing tasks. It outperforms other models, including the Llama 2 13B, across various benchmarks. Mistral 7B utilizes advanced attention mechanisms like grouped-query attention and sliding window attention to enhance inference speed and handle longer sequences effectively. It is released under the Apache 2.0 license and is suitable for a wide range of applications.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is mistral 7b?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Mistral 7B is a 7-billion-parameter language model designed for high performance and efficiency in natural language processing tasks. It outperforms other models, including the Llama 2 13B, across various benchmarks. Mistral 7B utilizes advanced 

In [109]:
question = "Tell me more about Mistral"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
display(Markdown(response['answer']))

Mistral 7B is engineered to balance high performance with efficiency, making it suitable for real-world applications. It employs innovative attention mechanisms, such as grouped-query attention (GQA) for faster inference and sliding window attention (SWA) to manage longer sequences with reduced computational costs. Mistral 7B has been fine-tuned to follow instructions, resulting in a variant called Mistral 7B – Instruct, which surpasses the Llama 2 13B – Chat model in performance. The model is released under the Apache 2.0 license, facilitating easy deployment and fine-tuning for various tasks.

In [110]:
question = "how is it different from Gemini"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
display(Markdown(response['answer']))

Mistral 7B and Gemini models differ primarily in their architecture and intended use cases. Mistral 7B focuses on high performance and efficiency with advanced attention mechanisms, while Gemini models, such as Gemini Pro and Nano, are designed for specific tasks like summarization and reading comprehension, with a focus on on-device deployment. Additionally, Gemini models vary in size, with Nano models being smaller and optimized for efficiency, whereas Mistral 7B is larger and aims for broader applicability across various NLP tasks.

In [113]:
question = "how is it different from gpt"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
display(Markdown(response['answer']))

Mistral 7B differs from GPT models primarily in its architecture and design focus. While both are transformer-based language models, Mistral 7B employs specific attention mechanisms like grouped-query attention and sliding window attention to improve efficiency and performance, especially for longer sequences. Additionally, Mistral 7B is optimized for lower computational costs and faster inference, aiming to balance high performance with efficiency, which may differ from the design goals of various GPT models.

## Multi-user Conversational RAG system with LangChain

In [ ]:
# you can run this only when you want to remove all conversation histories
#!rm memory.db

In [114]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI




In [115]:
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

######### REPHRASER ############
rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is."""


In [116]:
rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [117]:
history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)

In [142]:
######### MULTI_USER RAG RESPONSE GENERATOR ############
qa_system_prompt = """You are an assistant for question-answering tasks.
                      Use the following pieces of retrieved context to answer the question.
                      If you don't know the answer, just say that you don't know.
                      Keep the answer upto 5 lines unless the user asks for more information.
                      Also, search for the answer from RAG only


                      Context:
                      {context}
                  """

In [143]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [144]:
# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id, topk_conversations=2):
    history = SQLChatMessageHistory(session_id, "sqlite:///memory.db")
    #history.messages = history.messages[-2*topk_conversations:]
    return history

In [145]:
# subset historical conversations
def memory_buffer_window(messages, topk_conversations=2): # each conversation has 2 messages - (human prompt, AI response)
    return messages[-(topk_conversations*2):]

In [146]:
# custom RAG chain which looks at last K conversational messages
question_answer_chain = (
    RunnablePassthrough.assign(chat_history=lambda x: memory_buffer_window(x["chat_history"]),
                                context=lambda x: format_docs(x["context"])
                               )
      |
    qa_prompt
       |
    chatgpt
      |
    StrOutputParser()
)
qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [147]:

############ CONVERSATIONAL RAG CHAIN ####################
conversational_rag_chain = RunnableWithMessageHistory(
    qa_rag_chain,
    get_session_history_db,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [148]:
from IPython.display import display, Markdown

def conv_rag_chatbot(usersession_id, prompt):
    response = conversational_rag_chain.invoke(
                                {"input": prompt},
                                config={
                                    "configurable": {"session_id": usersession_id}
                                }
    )
    print('Answer:')
    display(Markdown(response['answer']))
    print('Sources:')
    for document in response['context']:
        print(document)
        print()

    return response

In [149]:
us_id = 'vij01'
r = conv_rag_chatbot(us_id, 'what is gpt')

Answer:


GPT, or Generative Pre-trained Transformer, is a type of artificial intelligence model developed by OpenAI. It is designed to understand and generate human-like text based on the input it receives. The model is pre-trained on a diverse range of internet text and can perform various language tasks, such as answering questions, summarizing text, and engaging in conversation. GPT-4, the latest version, is a multimodal model that can process both text and image inputs.

Sources:
page_content='GPT-4 Technical Report OpenAI∗ Abstract We report the development of GPT-4, a large-scale, multimodal model which can accept image and text inputs and produce text outputs. While less capable than humans in many real-world scenarios, GPT-4 exhibits human-level performance on various professional and academic benchmarks, including passing a simulated bar exam with a score around the top 10% of test takers. GPT-4 is a Transformer- based model pre-trained to predict the next token in a document. The post-training alignment process results in improved performance on measures of factuality and adherence to desired behavior. A core component of this project was developing infrastructure and optimization methods that behave predictably across a wide range of scales. This allowed us to accurately predict some aspects of GPT-4’s performance based on models trained with no more than 1/1,000th the compute of GPT-4. 1 Introduction This technical report presents GPT-4, a larg

In [151]:
us_id = 'vij01'
r = conv_rag_chatbot(us_id, 'what is the difference with mistral')

Answer:


Mistral 7B and GPT-4 are both advanced language models, but they differ in several key aspects:

1. **Architecture**: Mistral 7B uses a transformer architecture with innovations like grouped-query attention (GQA) and sliding window attention (SWA) for improved efficiency and performance, while GPT-4 is based on a standard transformer architecture.

2. **Model Size**: Mistral 7B has 7 billion parameters, whereas GPT-4's exact parameter count is not disclosed but is generally larger and more complex.

3. **Capabilities**: Mistral 7B is optimized for high performance in reasoning, mathematics, and code generation, while GPT-4 is multimodal, capable of processing both text and images.

4. **Release and Licensing**: Mistral 7B is released under the Apache 2.0 license, facilitating easier deployment, while GPT-4's access is typically through OpenAI's API and platforms.

Sources:
page_content='Mistral 7B Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed Abstract We introduce Mistral 7B, a 7–billion-parameter language model engineered for superior performance and efficiency. Mistral 7B outperforms the best open 13B model (Llama 2) across all evaluated benchmarks, and the best released 34B model (Llama 1) in reasoning, mathematics, and code generation. Our model leverages grouped-query attention (GQA) for faster inference, coupled with sliding window attention (SW A) to effectively handle sequences of arbitrary length with a reduced inference cost. We also provide a model fine-tuned to follow instructions, Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and autom

In [141]:
us_id = 'vij01'
r = conv_rag_chatbot(us_id, 'what is mistral')

/usr/local/lib/python3.11/dist-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='85076176-f6d8-4d01-88f9-784f968c7bba', metadata={'source': '/content/data/mistral_paper.pdf', 'page': 0}, page_content='Mistral 7B Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed Abstract We introduce Mistral 7B, a 7–billion-parameter language model engineered for superior performance and efficiency. Mistral 7B outperforms the best open 13B model (Llama 2) across all evaluated benchmarks, and the best released 34B model (Llama 1) in reasoning, mathematics, and code generation. Our model leverages grouped-query attention (GQA) for faster inference, coupled with sliding

Answer:


Mistral is a strong, cold wind that blows from the north or northwest in the Rhône valley of France. It is known for its dry and clear weather effects in the region. Additionally, "Mistral" can refer to a company or project in artificial intelligence, particularly related to developing advanced AI models. If you need more specific information, please let me know!

Sources:
